# imports

In [ ]:
from dataclasses import dataclass, field
import pygraphviz as pgv
import pandas as pd
INF = float('inf')

# решение

In [ ]:
@dataclass
class Graph:

    weights : dict[tuple[int, int], float]
    _verticies: tuple[str, ...] = field(default = (), init = False)
    _graph: pgv.AGraph | None = field(default = None, init = False)
    _adjacency_matrix: pd.DataFrame | None = field(default = None, init = False)
    _path_matrix: pd.DataFrame | None = field(default = None, init = False)
    _vert_matrix: pd.DataFrame | None = field(default = None, init = False)

    debug: bool = field(default = False, init = True)

    @property
    def graph(self):
        if self._graph is None:
            self._generate_graph()
        return self._graph

    @property
    def adjacency_matrix(self):
        if self._adjacency_matrix is None:
            self._generate_tables()
        return self._adjacency_matrix

    @property
    def path_matrix(self):
        if self._path_matrix is None:
            self._generate_tables()
        return self._path_matrix

    @property
    def vertices(self):
        if len(self._verticies) == 0:
            self._generate_tables()
        return self._verticies

    @property
    def vert_matrix(self):
        if self._vert_matrix is None:
            self._generate_tables()
        return self._vert_matrix
    
    def _generate_graph(self):
        ...

    # (1) По алгоритму Флойда-Уоршалла найти матрицу весов маршрутов И матрицу номеров
    # первых вершин этих маршрутов.
    def _generate_tables(self):
        
        vs = set[int]()
        d = dict[str, list[float]]()
        p = dict[str, list[str]]()
        for k, v in self.weights.items():
            vs |= set(k)
        self._verticies = tuple(map(lambda x: f'v{x}', sorted(vs)))
        length = max(vs)

        for j in range(1, length+1):
            d[f'v{j}'] = [INF]*length
            p[f'v{j}'] = [f'v{j}']*length
            d[f'v{j}'][j-1] = 0
        for k, v in self.weights.items():
            v1, v2 = tuple(k)
            d[f'v{v1}'][v2-1] = v
            d[f'v{v2}'][v1-1] = v
        for j in range(1, length+1):
            for i in range(length):
                if d[f'v{j}'][i] == INF:
                    p[f'v{j}'][i] = '0'

        self._adjacency_matrix = pd.DataFrame(d , index=[f'v{i}' for i in range(1, length+1)])

        L = self._adjacency_matrix.copy()
        P = pd.DataFrame(p , index=[f'v{i}' for i in range(1, length+1)])
        for k in range(length):
            if self.debug:
                print(f'L({k}) =')
                print(L)
                print(f'P({k}) =')
                print(P)
            for i in range(length):
                vi, vk = f'v{i+1}', f'v{k+1}'
                if i != k and L.loc[vi, vk] != INF:
                    for j in range(length):
                        vj = f'v{j+1}'
                        if j != k and L.loc[vk, vj] != INF:
                            T: float = L.loc[vi, vk] + L.loc[vk, vj] # type: ignore
                            if T == INF or T < L.loc[vi, vj]: # type: ignore
                                L.loc[vi, vj] = T
                                P.loc[vi, vj] = P.loc[vi, vk]
        if self.debug:
            print('Матрица смежности:')
            print(self.adjacency_matrix)
            print('Матрица весов маршрутов:')
            print(L)
            print('Матрица  номеров первых вершин этих маршрутов:')
            print(P)
        self._path_matrix = L
        self._vert_matrix = P

    # (2) Для указанной пары вершин vb ---> vt извлечь из матрицы маршрутов маршрут от
    # вершины vb к вершине vt в виде последовательности номеров вершин
    def get_path(self, vb: str, vt: str):
        path: list[str] = [vb]
        k = 0
        if self.debug:
            print(f"Поиск пути от вершины {vb} к вршине {vt} по матрице маршрутов")
            print(self.vert_matrix)
            print("Начальная вершина", vb)
            print("Текущий путь:", path)

        while path[-1] != vt:
            next_v = self.vert_matrix.loc[path[-1], vt] # type: ignore
            path.append(next_v) # type: ignore
            if self.debug:
                print("Следующая вершина", next_v)
                print("Текущий путь:", path)
                
        if self.debug:
            print("Итоговый путь:", path)

        return path
    
    # (3) По алгоритму Дейкстры для этой же пары вершин найти вес и маршрут 
    # минимального веса.
    def dijkstra(self, vb: str, vt: str):
        b = self.vertices.index(vb)
        t = self.vertices.index(vt)
        L = [INF] * len(self.vertices)
        P = ['-'] * len(self.vertices)
        C = [0] * len(self.vertices)
        k = 0
        Ms = [vb]
        T = pd.DataFrame({'L' : L, 'P': P, 'C': C}, index=self.vertices).T
        T.loc['L', vb] = 0
        T.loc['C', vb] = 1
        Ts = [T]
        if self.debug:
            print(f"T({k})")
            print(T)
            print(f"m({k})")
            print(vb)
        while (Ts[-1].loc['C'] == 0).any():
            k += 1
            T = Ts[-1].copy()
            Ts.append(T)
            for vi in self.vertices:
                if Ts[-1].loc['C', vi] == 0:
                    curr = Ts[-1].loc['L', Ms[-1]] + self.adjacency_matrix.loc[Ms[-1], vi] # type: ignore
                    if curr < Ts[-1].loc['L', vi]: # type: ignore
                        T.loc['L', vi] = curr
                        T.loc['P', vi] = Ms[-1] # self.verticies.index(Ms[-1])
            f1 = T.loc['C'] == 0
            f2 = T.loc['L'][f1] == T.loc['L'][f1].min()
            m = T.loc['L'][f1][f2].axes[0][0]
            T.loc['C', m] = 1
            if self.debug:
                print(f"T({k})")
                print(T)
                print(f"m({k})")
                print(m)
            Ms.append(m)
        T = Ts[-1]
        path = [vt]
        if self.debug:
            print(f"Добавить конечную вершину {vt} в путь: {path}")
        weight = T.loc['L', vt]
        while vb not in path:
            v = T.loc['P', path[0]]
            path.insert(0, v)
            if self.debug:
                print(f"Добавить вершину {v} в путь: {path}")
        
        if self.debug:
            print(f"Итоговый путь: {path}")
            print(f"Вес пути: {weight}")
        return weight, path

    # (4) По алгоритму Прима для данного графа найти минимальный остов в виде списка 
    # ребер остова.
    def find_island(self):
        vi = self.vertices[0]
        W = self.adjacency_matrix.copy()
        W[W==0] = INF

        not_selected = [False] + [True] * (len(self.vertices) - 1)
        edges = []
        total_weight = 0
        if self.debug:
            print('Матрица на начало')
            print(W)
        while len(edges) < (len(self.vertices) - 1):
            min_weight = float('inf')
            from_vertex = to_vertex = -1
            
            for i in range(len(self.vertices)):
                vi = f'v{i+1}'
                if not not_selected[i]:
                    for j in range(len(self.vertices)):
                        vj = f'v{j+1}'
                        if not_selected[j] and W.loc[vi, vj] > 0:
                            if W.loc[vi, vj] < min_weight:
                                min_weight = W.loc[vi, vj]
                                from_vertex = i
                                to_vertex = j
            
            vb = self.vertices[from_vertex]
            vt = self.vertices[to_vertex]
            if from_vertex != -1 and to_vertex != -1:
                w = min_weight
                edges.append(f'W{set((vb, vt))}={w}')
                not_selected[to_vertex] = False
                total_weight += min_weight
            if self.debug:
                print(f'Построено ребро из вершины {vb} в вершину {vt}')
                print('Матрица')
                print(W.loc[:, not_selected])
                print(f'Рёбра: {edges}')
        
        return edges, total_weight

In [ ]:
var1 = {
     tuple({1,2}):11, tuple({1,7}):10, tuple({2,3}):31, tuple({2,6}):25, tuple({2,8}):21, 
tuple({3,5}):3, tuple({3,6}):16, tuple({4,5}):20, tuple({4,6}):14, tuple({5,8}):22, 
tuple({6,7}):19, tuple({6,8}):1, tuple({7,8}):5
}

In [ ]:
v1 = Graph(var1, True)
# print(v1.adjacency_matrix)
# print(v1.path_matrix)
# print(v1.vert_matrix)
# print(v1.weights)

In [ ]:
e, w = v1.find_island()

print(e)
print(w)
# w.loc[:, 'v1']=INF
# print(w)
# v = w.loc['v1'][w.loc['v1']==w.loc['v1'].min()].axes[0][0]
# w.loc[:, v] = INF
# print(w)
# w.loc[:, [False] + [True]*7]

In [ ]:
# v4 -> v7
print(v1.get_path('v4', 'v7'))

In [ ]:
t = v1.dijkstra('v4', 'v7')
print(len(t), *t, sep='\n-\n')

In [ ]:
v1.vertices[2,3]